# CE40719: Deep Learning
## HW3  


#### Name:
#### Student No.:

# Part 1. NumPy Based CNN block (15+5 Points)

### Deadline:   

In [6]:
import numpy as np
import os
import matplotlib.pyplot as plt
from CE40719.module import *
from CE40719.batch_norm import *
from CE40719.cnn_batch_norm import *
from CE40719.convolution import *
from CE40719.group_batch_norm import *
from CE40719.max_pool import *


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##   Outline of the Assignment

You will be implementing the building blocks of a convolutional neural network
    
This notebook will ask you to implement these functions from scratch in **`Numpy`**.
 
1. **`convolution `** 

2. **`Max pooling`** 

3. **`CNN Batch Normalization`**: Batch Normalization layer in cnn

4. **`Group Normalization`**: layer which normalize over the group of feature per-datapoint(E.g. group of pixel per-image on first layer of network)


In this problem you have to implement `forward()` and `backward()` functions of the modules in the above list. Then your implemented codes will be tested here in this notebook.

# Convolution (7 Points)
Open file `convolution.py`. There you see a class for convolutione layer used in CNN networks.<br/> 

Implement `forward()` and `backward()` functions corresponding to forward and backward pass of a concolution layer.<br/>
We have implemented `__init__()` constructor for you. You can see `W` with shape of $(F, C, f_h, f_w)$, `b` with shape of $(F,)$, `dW`, `db`, `stride` number of pixels between adjacent receptive fields in the horizontal and vertical directions and `pad` number of pixels that will be used to zeropad the input variables in this class.<br/>
The input of forward pass is X with shape of $(N, C, i_h, i_w)$, The return value of forward pass is o the shape is $(N, F, O_h, O_w)$ where 

$$\text{stride} : int $$

$$\text{pad} : int$$

$$O_w =\lfloor \frac{i_w - f_w + 2*pad}{stride} \rfloor + 1$$

$$O_h = \lfloor\frac{i_h - f_h + 2*pad}{stride}\rfloor + 1$$
$$O(b,f, i ,j)=\sum_{r=0}^{C-1}\sum_{k=0}^{f_h-1}\sum_{l=0}^{f_w-1} W(f,r,k,l) X(b,r,s_h *i +k, s_w  *j +l)+b(f)$$.<br/>
In the backward pass you get $\large{\frac{\partial{loss}}{\partial{y}}}$ as `dout`; You have to compute $\large{\frac{\partial{loss}}{\partial{X}}}$, $\large{\frac{\partial{loss}}{\partial{W}}}$ and $\large{\frac{\partial{loss}}{\partial{b}}}$ and save them in `dx`, `dW`, and `db`. The output of the `backward()` function will be `dx`.<br/>
This is the formula for computing a $\frac{\partial L}{\partial X}$ for a single $X(b^\prime,c^\prime,k^\prime ,l^\prime )$ that $X$ is 4-D array as a input in convolution operation with shape $(N,C,i_h,i_w)$

$$\frac{\partial L}{\partial X(b^\prime,c^\prime,k^\prime ,l^\prime )} = \sum_{f=0}^{F-1}\left (\sum_{i=0}^{O_h-1}\sum_{j=0}^{O_w-1} \frac{\partial L}{\partial O(b^\prime,f,i,j)} \frac{\partial O(b^\prime,f,i,j)}{\partial X(b^\prime,c^\prime,k^\prime ,l^\prime )}\right ) = \sum_{f=0}^{F-1}\left (\sum_{i=0}^{O_h-1}\sum_{j=0}^{O_w-1} \frac{\partial L}{\partial O(b^\prime,f,i,j)} W(f,c^\prime,k^\prime - s_h*i, l^\prime - s_w*j) \right )$$<br/>

 This is the formula for computing a $\frac{\partial L}{\partial W}$ for a single $W(f^\prime,c^\prime,k^\prime ,l^\prime )$ that $W$ is 4-D array as a filter in convolution operation with shape $(F,C,f_h,f_w)$
 
 $$\frac{\partial L}{\partial W(f^\prime,c^\prime,k^\prime ,l^\prime )} = \sum_{b=0}^{N-1}\left (\sum_{i=0}^{O_h-1}\sum_{j=0}^{O_w-1} \frac{\partial L}{\partial O(b,f^\prime,i,j)} \frac{\partial O(i,j)}{\partial W(f^\prime,c^\prime,k^\prime ,l^\prime )}\right ) = \sum_{b=0}^{N-1}\left (\sum_{i=0}^{O_w-1}\sum_{j=0}^{O_h-1} \frac{\partial L}{\partial O(b,f^\prime,i,j)}  X(b,c^\prime, s_h*i +k^\prime, s_w*j +l^\prime) \right )$$<br/>
 the formula for $\large{\frac{\partial{loss}}{\partial{b}}}$ is left for you to compute.

In [2]:
###########################################################################
#                         Convolution Test                                #
###########################################################################
np.random.seed(40959)
x = np.random.randn(2, 3, 4, 4)
w = np.random.randn(3, 3, 4, 4)
b = np.random.randn(3,)
conv = Convolution('test', (3, 3, 4, 4), stride = 2, pad = 1)
conv.W = w
conv.b = b
output = conv.forward(x)
correct_output = np.array([[[[2.90745973, -1.93569447],
                           [10.18479326, 0.95405206 ]],
                          [[ 2.67941204,  -1.06629218],
                           [ -2.6427213,  -3.12561258]],
                          [[ -5.32408346,  1.12473438],
                           [ 4.16451343,  -5.04230883]]],
                         [[[0.18517581, 10.22485798],
                           [-3.51174763, 1.9202936]],
                          [[ -2.56595929,  -3.40545467],
                           [ 0.33082083,  4.34434771]],
                          [[ -3.54337648,  2.44988087],
                           [ -3.6369818,  1.96857427]]]])

print('Relative error forward pass:', np.linalg.norm(output - correct_output))


conv = Convolution('test', (2, 3, 2, 2), stride = 2, pad = 1)
x = np.random.randn(2, 3, 2, 2)
W = np.random.randn(2, 3, 2, 2)
b = np.random.randn(2,)

conv.W = W
conv.b = b
dout = np.random.randn(2, 2, 2, 2)
out = conv.forward(x)
dx= conv.backward(dout)


correct_dx = np.array([[[[-0.03022149, -0.93652977],[-0.05179407, 1.62558139]],
                        [[ 1.62356625,  3.17432728],[1.37585703,  0.21801443]],
                        [[ -1.14110006,  -3.2751212],[0.98650008,   0.78396852 ]]],
                       [[[0.48556001,  1.24240355],[ 0.1635526,  0.97860699]],
                        [[2.07933521, -1.62650629],[ -0.35726596, 0.17660094]],
                        [[ 0.27806844,  2.30231871],[-1.07156607, 0.22142858]]]])
print('Relative error dx:', np.linalg.norm(dx - correct_dx))
    
correct_dW =np.array([[[[0.16162545,  0.44372442],[ 0.5131281 , 0.41785749]],
                       [[-0.4409529, -0.31301584],[-0.18734267,  0.06869406]],
                       [[ -0.53426167, -0.94735183],[ 0.9614619, 0.36417281]]],
                      [[[ -0.40656537, -0.1906337  ],[1.38892306, -0.59866861]],
                       [[ 0.81392044, 0.36665929 ],[0.78840142, 2.80736748]],
                       [[1.58139656, -0.81670389],[ -1.11075549,  -1.7656368]]]])
print('Relative error dw:', np.linalg.norm(conv.dW - correct_dW))
    
correct_db =np.array([0.82375129, 2.84032899])
print('Relative error db:', np.linalg.norm(conv.db - correct_db))

Relative error forward pass: 5.014143567861652e-08


TypeError: unsupported operand type(s) for -: 'NoneType' and 'float'

** Expected Output: **
<table>    
    <tr>
        <td>
            **Relative error forward pass**
        </td>
        <td>
            5.0141436175352257e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error dx**
        </td>
        <td>
            1.3685826529800791e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error dw**
        </td>
        <td>
            1.3702887296620126e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error db**
        </td>
        <td>
            2.2920873066590985e-09
        </td>
    </tr>
</table>

# Max-Pooling (5 Points)
The pooling layer reduces the height and width of the input. It helps reduce computation<br/>
Max-pooling layer slides an ($f, f$) window with stride $s$ over the input and stores the max value of the window in the output.
Open file `max_pool.py`. There you see a class for pooling layer.<br/> 

We have implemented `__init__()` constructor for you. Similar to `Convolution` class, there are pool hight,  pool width and stride variables in this class.

Implement `forward()` and `backward()` functions corresponding to forward and backward pass of pooling layer. Even though a pooling layer has no parameters for backprop to update, you still need to backpropagation the gradient through the pooling layer in order to compute gradients for layers that came before the pooling layer.<br/>
in `forward()` function X is input and shape of X is $(N, C, i_h, i_w)$  and output is shape $(N, C, O_h, O_w)$ that :

 $$O_h =\lfloor\frac{i_h - f }{s}\rfloor + 1$$
 $$O_w =\lfloor\frac{i_w - f }{s}\rfloor + 1$$

 No padding is necessary here

In [7]:
###########################################################################
#                            Max pool test                                #
###########################################################################
np.random.seed(40959)
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.3, 0.4, num=np.prod(x_shape)).reshape(x_shape)
pool = MaxPool('test',height = 2,width = 2, stride = 2)
output = pool.forward(x)

correct_output = np.array([[[[-0.26315789, -0.24842105],
                          [-0.20421053, -0.18947368]],
                         [[-0.14526316, -0.13052632],
                          [-0.08631579, -0.07157895]],
                         [[-0.02736842, -0.01263158],
                          [ 0.03157895,  0.04631579]]],
                        [[[ 0.09052632,  0.10526316],
                          [ 0.14947368,  0.16421053]],
                         [[ 0.20842105,  0.22315789],
                          [ 0.26736842,  0.28210526]],
                         [[ 0.32631579,  0.34105263],
                          [ 0.38526316,  0.4       ]]]])

print('Relative error forward pass:', np.linalg.norm(output - correct_output))

x = np.random.randn(3, 2, 2, 2)
dout = np.random.randn(3, 2, 1, 1)
pool = MaxPool('test',height = 2, width = 2, stride =2)
out = pool.forward(x)
dx = pool.backward(dout)
correct_dx = np.array([[[[0., 1.21236066],[0., 0.]],
                        [[0.45107133, 0.],[0., 0.]]],
                       [[[0., 0.],[-0.86463156,0.]],
                        [[0., 0.],[0., -0.39180953]]],
                       [[[0.93694169, 0.],[0., 0.]],
                        [[0., 0.],[-0.08002411, 0.]]]])
print('Relative error dx:', np.linalg.norm(dx - correct_dx))


Relative error forward pass: 1.4161709469389779e-08
Relative error dx: 8.555596854220575e-09


** Expected Output: **
<table>    
    <tr>
        <td>
            **Relative error forward pass**
        </td>
        <td>
            4.1666665157267834e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error dx**
        </td>
        <td>
            8.555596854220575e-09
        </td>
    </tr>
</table>

## CNN batch normalization(3 Points)
Normally batch-normalization accepts inputs of shape `(N, D)` and produces outputs of shape `(N, D)`, where we normalize across the minibatch dimension `N`. For data coming from convolutional layers, batch normalization needs to accept inputs of shape $(F, C, f_h, f_w)$ and produce outputs of shape $(F, C, f_h, f_w)$ where the `N` dimension gives the minibatch size and the $(f_h, f_w)$ dimensions give the spatial size of the feature map.

If the feature map was produced using convolutions, then we expect every feature channel's statistics e.g. mean, variance to be relatively consistent both between different images, and different locations within the same image -- after all, every feature channel is produced by the same convolutional filter! Therefore spatial batch normalization computes a mean and variance for each of the `C` feature channels by computing statistics over the minibatch dimension `N` as well the spatial dimensions  $f_h$ and  $f_h$.

Open file `cnn_batch_norm.py`. There you see a class for CNN Batch Normalization layer.<br/> 

Implement `forward()` and `backward()` functions corresponding to forward and backward pass of cnn Batch Normalization layer.<br/>
 
We have implemented `__init__()` constructor for you. Similar to `Convolution` class, there are `gamma`, `beta`, `dgamma` and `dbeta` variables in this class. also there is one auxiliary variable `batchnorm` from previous assignment witch will help you to implement cnn batchnormalization easily.
Test your implementations with the following cell.

In [ ]:
###########################################################################
#            Batch-Normalization forward pass Test                        #
###########################################################################
np.random.seed(40959)
# Check the training-time forward pass by checking means and variances
# of features both before and after spatial batch normalization

N, C, H, W = 2, 3, 4, 5
x = 4 * np.random.randn(N, C, H, W) + 10

print('Before spatial batch normalization:')
print('  Shape: ', x.shape)
print('  Means: ', x.mean(axis=(0, 2, 3)))
print('  Stds: ', x.std(axis=(0, 2, 3)))

# Means should be close to zero and stds close to one
gamma, beta = np.ones(C), np.zeros(C)
cnn_batchnorm = CnnBatchNorm('test',(N, C, H, W))
cnn_batchnorm.batchnorm.gamma = gamma
cnn_batchnorm.batchnorm.beta = beta
out = cnn_batchnorm.forward(x)
print('After spatial batch normalization:')
print('  Shape: ', out.shape)
print('  Means: ', out.mean(axis=(0, 2, 3)))
print('  Stds: ', out.std(axis=(0, 2, 3)))

# Means should be close to beta and stds close to gamma
gamma, beta = np.asarray([3, 4, 5]), np.asarray([6, 7, 8])
cnn_batchnorm.batchnorm.gamma = gamma
cnn_batchnorm.batchnorm.beta = beta
out= cnn_batchnorm.forward(x)
print('After spatial batch normalization (nontrivial gamma, beta):')
print('  Shape: ', out.shape)
print('  Means: ', out.mean(axis=(0, 2, 3)))
print('  Stds: ', out.std(axis=(0, 2, 3)))

In [ ]:
np.random.seed(40959)
# Check the test-time forward pass by running the training-time
# forward pass many times to warm up the running averages, and then
# checking the means and variances of activations after a test-time
# forward pass.
N, C, H, W = 10, 4, 11, 12
cnn_batchnorm = CnnBatchNorm('test',(N, C, H, W))
cnn_batchnorm.train()
for t in range(50):
  x = 2.3 * np.random.randn(N, C, H, W) + 13
  cnn_batchnorm.forward(x)
cnn_batchnorm.test()
x = 2.3 * np.random.randn(N, C, H, W) + 13
a_norm = cnn_batchnorm.forward(x)

# Means should be close to zero and stds close to one, but will be
# noisier than training-time forward passes.
print('After spatial batch normalization (test-time):')
print('  means: ', a_norm.mean(axis=(0, 2, 3)))
print('  stds: ', a_norm.std(axis=(0, 2, 3)))

In [ ]:
###########################################################################
#            Batch-Normalization backward pass Test                       #
###########################################################################
np.random.seed(40959)
N, C, H, W = 2, 3, 2, 2
x = 5 * np.random.randn(N, C, H, W) + 12
gamma = np.random.randn(C)
beta = np.random.randn(C)
dout = np.random.randn(N, C, H, W)
cnn_batchnorm = CnnBatchNorm('Test',(N, C, H, W))
cnn_batchnorm.batchnorm.gamma = gamma
cnn_batchnorm.batchnorm.beta = beta
cnn_batchnorm.train()
_ = cnn_batchnorm.forward(x)
dx = cnn_batchnorm.backward(dout)
dgamma = cnn_batchnorm.dgamma
dbeta = cnn_batchnorm.dbeta
correct_dx = np.array([[[[0.00589789,  1.2557341 ],[-0.18515455, -0.3084614 ]],
                        [[-0.04023214, -0.11912787],[-0.04556006, -0.00270806]],
                        [[ 0.12266522, -0.07093585],[ 0.22957267,  0.17611092]]],
                       [[[ 0.36047414, -0.01314037],[-0.62981818, -0.48553163]],
                        [[ 0.18630326, -0.02134853],[-0.15169621,  0.19436962]],
                        [[-0.00739465, -0.04518148],[-0.2105455,  -0.19429132]]]])
print('Relative error dx:', np.linalg.norm(dx - correct_dx))
correct_dgamma = np.array([ 1.51945006, -1.09337409,  0.8928227 ])
print('Relative error dgamma:', np.linalg.norm(dgamma - correct_dgamma))
correct_dbeta = np.array([-3.1690584,   3.01154949,  5.44132887])
print('Relative error dbeta:', np.linalg.norm(dbeta - correct_dbeta))

** Expected Output: **
<table>    
    <tr>
        <td>
            **Relative error dx**
        </td>
        <td>
            1.2516125487116332e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error dgamma**
        </td>
        <td>
            4.2623728399775465e-09
        </td>
     </tr>
    <tr>
        <td>
            **Relative error dbeta**
        </td>
        <td>
            6.5396326216333006e-09
        </td>
     </tr>    
</table>

# Group Normalization (5 extra Points)
Batch normalization has proved to be effective in making networks easier to train, but the dependency on batch size makes it less useful in complex networks which have a cap on the input batch size due to hardware limitations. 

Several alternatives to batch normalization have been proposed to mitigate this problem; one such technique is Layer Normalization [1]. Instead of normalizing over the batch, we normalize over the features. In other words, when using Layer Normalization, each feature vector corresponding to a single datapoint is normalized based on the sum of all terms within that feature vector.

Layer Normalization mitigates the batch size limitations of Batch Normalization. However, as the authors of [2] observed, Layer Normalization does not perform as well as Batch Normalization when used with Convolutional Layers:

>With fully connected layers, all the hidden units in a layer tend to make similar contributions to the final prediction, and re-centering and rescaling the summed inputs to a layer works well. However, the assumption of similar contributions is no longer true for convolutional neural networks. The large number of the hidden units whose
receptive fields lie near the boundary of the image are rarely turned on and thus have very different
statistics from the rest of the hidden units within the same layer.

The authors of [2] propose an intermediary technique. In contrast to Layer Normalization, where you normalize over the entire feature per-datapoint, they suggest a consistent splitting of each per-datapoint feature into G groups, and a per-group per-datapoint normalization instead. 

![Comparison of normalization techniques discussed so far](notebook_images/normalization.png)
<center>**Visual comparison of the normalization techniques discussed so far (image edited from [2])**</center>

Even though an assumption of equal contribution is still being made within each group, the authors hypothesize that this is not as problematic, as innate grouping arises within features for visual recognition. One example they use to illustrate this is that many high-performance handcrafted features in traditional Computer Vision have terms that are explicitly grouped together. Take for example Histogram of Oriented Gradients [3]-- after computing histograms per spatially local block, each per-block histogram is normalized before being concatenated together to form the final feature vector.

You will now implement Group Normalization. Note that this normalization technique that you are to implement in the following cells was introduced and published to ECCV  in 2018 .

[1] [Ba, Jimmy Lei, Jamie Ryan Kiros, and Geoffrey E. Hinton. "Layer Normalization." stat 1050 (2016): 21.](https://arxiv.org/pdf/1607.06450.pdf)


[2] [Wu, Yuxin, and Kaiming He. "Group Normalization." arXiv preprint arXiv:1803.08494 (2018).](https://arxiv.org/abs/1803.08494)


[3] [N. Dalal and B. Triggs. Histograms of oriented gradients for
human detection. In Computer Vision and Pattern Recognition
(CVPR), 2005.](https://ieeexplore.ieee.org/abstract/document/1467360/)

## Group normalization

Open file `group_batch_norm.py`. There you see a class for group Batch Normalization layer.<br/> 

Implement `forward()` and `backward()` functions corresponding to forward and backward pass of group Batch Normalization layer.<br/>
 
We have implemented `__init__()` constructor for you. Similar to `Convolution` class, there are `G` integer number of groups to split into which should be a divisor of number of channel, `gamma`, `beta`, `dgamma` and `dbeta` variables in this class. 
Test your implementations with the following cell.

In [ ]:
###########################################################################
#            Group-Normalization forward Test                             #
###########################################################################
np.random.seed(40959)
# Check the training-time forward pass by checking means and variances
# of features both before and after spatial batch normalization

N, C, H, W = 2, 6, 4, 5
G = 2
x = 4 * np.random.randn(N, C, H, W) + 10
x_g = x.reshape((N*G,-1))
print('Before spatial group normalization:')
print('  Shape: ', x.shape)
print('  Means: ', x_g.mean(axis=1))
print('  Stds: ', x_g.std(axis=1))

# Means should be close to zero and stds close to one
norm = GroupBatchNorm('test',(N, C, H, W), G=2)
out = norm.forward(x)
out_g = out.reshape((N*G,-1))
print('After spatial group normalization:')
print('  Shape: ', out.shape)
print('  Means: ', out_g.mean(axis=1))
print('  Stds: ', out_g.std(axis=1))

In [ ]:
###########################################################################
#            Group-Normalization backward Test                            #
###########################################################################
np.random.seed(40959)
N, C, H, W = 2, 4, 2, 2
G = 2
x = 5 * np.random.randn(N, C, H, W) + 12
dout = np.random.randn(N, C, H, W)
norm = GroupBatchNorm('test',(N, C, H, W), G)
_ = norm.forward(x)
dx = norm.backward(dout)
dgamma = norm.dgamma
dbeta = norm.dbeta
correct_dx = np.array([[[[-0.34315175, -0.12381128],[ 0.03543989, -0.10811583]],
                        [[ 0.23704048,  0.26681368],[-0.35225047,  0.38803527]],
                        [[ 0.08007765, -0.00905627],[-0.05405272, -0.10643134]],
                        [[-0.09270617, -0.26378193],[ 0.36204868,  0.08390209]]],
                       [[[-0.15694482,  0.13000584],[-0.10158775, -0.07278944]],
                        [[ 0.29256627,  0.00665728],[-0.08065789, -0.0172495 ]],
                        [[-0.01647735,  0.09978986],[-0.11498191,  0.05169455]],
                        [[ 0.09743741, -0.18717901],[ 0.12091729, -0.05120085]]]])
print('Relative error dx:', np.linalg.norm(dx - correct_dx))
correct_dgamma = np.array([[[[-3.28150076]],[[-5.58372413]],[[ 2.98735869]],[[ 3.02274058]]]])
print('Relative error dgamma:', np.linalg.norm(dgamma - correct_dgamma))
correct_dbeta = np.array([[[[ 0.28621793]],[[ 5.8519097 ]],[[-3.48103948]],[[-2.95280322]]]])
print('Relative error dbeta:', np.linalg.norm(dbeta - correct_dbeta))

** Expected Output: **
<table>    
    <tr>
        <td>
            **Relative error dx**
        </td>
        <td>
            1.4961328597826837e-08
        </td>
    </tr>
    <tr>
        <td>
            **Relative error dgamma**
        </td>
        <td>
            5.694257120494842e-09
        </td>
     </tr>
    <tr>
        <td>
            **Relative error dbeta**
        </td>
        <td>
            4.410581982727471e-09
        </td>
     </tr>    
</table>